# MATH10098: Numerical Linear Algebra - Computer lab week 5

Below is an implementation of Algorithm GEPP. Have a look at the code to make sure you understand how it works before moving on to the exercises.

In [4]:
# Implementation of Algorithm GEPP.

import numpy as np

# Using forwardsub and backsub from week 3.

def FS(L, b):
    # Find dimension of L
    n = L.shape[0]
    
    # Initialise y to zero vector of correct length
    y = np.zeros(n)
    
    # loop over elements of y, starting from first one
    for j in range(n):
        y[j] = (b[j] - L[j, :j] @ y[:j]) / L[j, j] # compute y_j
        
    return y

def BS(U, y):
    # Find dimension of U
    n = U.shape[0]
    
    # Initialise x to zero vector of correct length
    x = np.zeros(n)
    
    # loop over elements of x, starting from last one
    for j in range(n-1, -1, -1):
        x[j] = (y[j] - U[j, j+1:] @ x[j+1:]) / U[j, j] # compute x_j
    
    return x

# New implementation of Algorithm LUPP

def LUPP(A):
    # Find dimension of A
    n = A.shape[0]
    
    # Initialise matrices L=I, U=A, P=I
    L = np.eye(n)
    U = np.copy(A)
    P = np.eye(n)
    
    for k in range(n - 1): # Loop over columns 1 to n-1
        
        # Find pivot i
        i = np.argmax(np.abs(U[k:, k])) # Find position of maximum entry in required range
        i += k # Shift index to start counting from row 1
        
        # Swap rows correspondingly
        if i != k:
            # swap rows in U, columns k to n
            U[[k, i], k:] = U[[i, k], k:]
            
            # swap rows in L, columns 1 to k-1
            L[[k, i], :k] = L[[i, k], :k]
            
            # swap rows in P
            P[[k, i], :] = P[[i, k], :]
        
        # Loop over rows and make entries below
        # diagonal in column k equal to 0
        for j in range(k + 1, n): # loop over rows k+1 to n
            L[j, k] = U[j, k] / U[k, k] # compute the multiplier l_jk
            U[j, k:] = U[j, k:] - L[j, k] * U[k, k:] # subtract a multiple of row k from row j
        
    return L, U, P

def GEPP(A, b):
    L, U, P = LUPP(A) # find LU factorisation with permuation of A
    y = FS(L, P @ b) # solve Ly=Pb using forward substitution
    x = BS(U, y) # solve Ux=y using back substitution
    return x

# Test the functions
A = np.array([[2, 1, 1], [4, 3, 3], [8, 7, 9]], dtype=float)
b = np.array([4, 10, 24], dtype=float)
x = GEPP(A, b)
print('x = {}'.format(x))

x = [1. 1. 1.]


---

### Exercise 1 (Easy)

You may wish to use functions you have written in previous workshops and/or functions that were presented in lectures.

For $n=50 \times 2^k$ with $k=1, 2, \dots, 5$:

(i) Set up a random matrix `A` of dimension $n$.

(ii) Create a vector `xsol` to represent a vector $x^{\ast} \in \mathbb{R}^n$, all of whose entries are $1$.

(iii) Compute the vector `b` such that $b = Ax^\ast$.

(iv) Use Algorithm GE to solve $Ax=b$, and store the solution in the variable `x1`. Take note of the elapsed time, as well as the $\infty-$norm of the residual `A @ x1 - b` and the error `xsol - x1`. 

(v) Use Algorithm GEPP to solve $Ax=b$, and store the solution in the variable `x2`. Take note of the elapsed time, as well as the $\infty-$norm of the residual `A @ x2 - b` and the error `xsol - x2`.

How do the elapsed times of the two algorithms compare? Is this what you expected? Do you notice any trends with n?

How do the norms of the residual and the error compare? Is this what you expected? Do you notice any trends with n?

In [5]:
import numpy as np
def LU(A):
    # Find dimension of A
    n = A.shape[0]
    
    # Initialise L=I, U=A
    L = np.eye(n) 
    U = np.copy(A)

    for k in range(n - 1): # loop over columns 1 to n-1
        for j in range(k + 1, n): # loop over rows k+1 to n
            L[j, k] = U[j, k] / U[k ,k] # compute the multiplier l_jk
            U[j, k:] = U[j, k:] - L[j, k] * U[k, k:] # subtract a multiple of row k from row j
    
    return L, U

def GE(A, b):
    L, U = LU(A)
    y = FS(L, b)
    x = BS(U, y)
    return x

In [11]:
import numpy as np
import time

# Initialise lists to store times
t_GEPP = []
t_GE = []
ge_norm_of_diff = []
gepp_norm_of_diff = []

for k in range (1,6):
    # init matrices
    A = np.random.rand(50*2**k, 50*2**k)
    xsol = np.random.rand(50*2**k)
    b = A @ xsol
    #start time
    start_time = time.time()
    x = GE(A, b)
    t_GE.append(time.time() - start_time)
    # norm of diff
    ge_norm_of_diff.append(np.linalg.norm(xsol - x))

    # GEPP
    start_time = time.time()
    x = GEPP(A, b)
    t_GEPP.append(time.time() - start_time)
    # norm of diff
    gepp_norm_of_diff.append(np.linalg.norm(xsol - x))


print(t_GE)
    

[0.045772552490234375, 0.14466404914855957, 0.38098812103271484, 1.5008656978607178, 7.05253791809082]


In [12]:
print(ge_norm_of_diff)
print(gepp_norm_of_diff)

[5.852067485640262e-12, 3.650141864189309e-11, 1.8517226000589403e-10, 1.6550859901134115e-08, 4.745268698433236e-09]
[1.4385651643615627e-13, 2.7189764757014477e-12, 3.2048356085653496e-12, 2.674651077530405e-11, 4.0998939421168116e-11]


---

### Exercise 2 (Standard)

Consider the system of linear equations
$$
\mathbf A \mathbf x = \mathbf b,
$$
where $\mathbf A \in \mathbb R^{n \times n}$ is invertible and $\mathbf b \in \mathbb R^n$.

Cramer's rule states that the solution is given by
$$
x_i = \frac{det (\mathbf {A_i})}{det(\mathbf A)}, \qquad i=1, \dots, n, 
$$
where $\mathbf {A_i} \in \mathbb R^{n \times n}$ is the matrix $\mathbf A$ with the $i$th column replaced by $\mathbf b$ and $det$ denotes the determinant.

(i) Write a function *det* that uses the LU factorisation of $\mathbf A$ to compute $det(\mathbf A)$. Test your code on the example at the bottom of the code cell (the correct answer is 2).

Have you thought about computational efficiency in your code? Ask your tutor to have a look.

*Python's built in determinant function np.linalg.det is in fact based on an LU factorisation with partial pivoting.*

In [36]:
import numpy as np


def det(A):  
    L, U = LU(A)
    d = 1
    for i in range(L.shape[0]):
        d *= L[i][i] * U[i][i]
    return d


# Testing the function on the given example, the correct answer is 2
A = np.array([[2, 1, 1], [4, 3, 3], [6, 7, 8]], dtype=float)
d = det(A)
print(d)

2.0


(ii) Write a function *solve_Cramer* that computes the solution $\bf x$ using Cramer's rule. You can use your function *det* from part (i), or the built-in Python routine np.linalg.det.

Have you thought about computational efficiency in your code? Ask your tutor to have a look.

In [34]:
A = np.ones((3,3))
b = 2*np.ones(3)
A[:,1]=b
A

array([[1., 2., 1.],
       [1., 2., 1.],
       [1., 2., 1.]])

In [41]:
import numpy as np

def solve_Cramer(A,b):
    aLen = A.shape[0]
    x = np.empty(aLen)
    for i in range(aLen):
        A_i = A.copy()
        A_i[:,i] = b
        x[i] = det(A_i)/det(A)
    
    return x

# Testing the function on the given example, the correct answer is [1, -2, 4]
A = np.array([[2, 1, 1], [4, 3, 3], [6,7,8]], dtype=float)
b = np.array([4,10,24], dtype=float) 
x = solve_Cramer(A,b)
print(x)

[ 1. -2.  4.]


(iii) How does Cramer's rule from part (ii) compare to the built-in Python solver np.linalg.solve in terms of computational time and accuracy of the computed solution? Test the two methods on matrices of varying size $n$ between 10 and 150.

In [ ]:
import numpy as np
import time


# add code here

---

### Exercise 3 (Standard, open-ended)

The accuracy of the computed solution $x$ of the system $Ax=b$ can sometimes be improved by a process called iterative refinement:

1. Solve $Ax = b$ with GE
2. Compute the residual $r = b - Ax$
3. Solve $Ad = r$
4. Update $x \leftarrow x + d$.

Steps 2--4 are repeated until convergence to a more accurate solution. At each iteration, since the matrices $L$, $U$ have already been computed when solving $Ax=b$, they can be used to solve $Ad=r$ at a cost of $O(n^2)$.

Write a function `GE_ref`, based on your function `GE`, to compute a solution to $Ax=b$ using GE with iterative refinement. Investigate the performance of `GE_ref` compared to that of `GE` and `GEPP`; you may wish to use test cases from Exercise 1 (investigating different $n$) and the Week 3.2 Jupyter notebook (investigating different condition numbers $\kappa_2(A)$).

Further improvements on the iterative refinement method are described in [this paper](https://doi.org/10.1137/17M1122918) -- you may wish to implement/investigate some of these.

In [ ]:
import numpy as np

# add code here

---

### Exercise 4 (Difficult)

The matrix $A \in \mathbb{R}^{n\times n}$ is called *tridiagonal* if $a_{ij}=0$ for $|i - j|>1$. Tridiagonal matrices appear naturally in many applications, such as the numerical solution of ordinary differential equations. Due to their simple structure, tridiagonal matrices have some special properties and specialised algorithms.

For a tridiagonal matrix $A$, the LU factors $L$ and $U$ are also tridiagonal:

$$
\begin{bmatrix}
a_{11} & a_{12} & & \\
a_{21} & a_{22} & \ddots &   \\
& \ddots & \ddots  & a_{n-1,n} \\
& & a_{n,n-1}& a_{nn}
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & & \\ l_{21} & 1 & \ddots &   \\
& \ddots & \ddots  & 0 \\
& & l_{n,n-1}& 1
\end{bmatrix}
\;
\begin{bmatrix}
u_{11} & u_{12} & & \\
0 & u_{22} & \ddots &   \\
& \ddots & \ddots  & u_{n-1,n} \\
& & 0& u_{nn}
\end{bmatrix}.
$$

(i) Explain why algorithm LU is not efficient for tridiagonal matrices, i.e. explain why it performs many unnecessary arithmetic operations.

(ii) Write down pseudo-code for an Algorithm LU-TRI, an efficient algorithm for computing the LU factorisation of a tridiagonal matrix. *Hint: Multiply the above matrices $L$ and $U$ to find equations for $u_{ii}$, $u_{i, i+1}$, and $l_{i, i-1}$ involving the entries of A. Start by computing $u_{11}$.*

(iii) Write a function `LU_tri` which implements your LU-TRI algorithm.

(iv) How does the computational time of Algorithm LU-TRI seem to grow with $n$? Is this what you would expect? You may wish to verify your findings numerically by setting up tridiagonal matrices using the command np.diag.

In [ ]:
import numpy as np

# add code here